In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
sys.path.insert(0, '../scripts')

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout
from mfcc_feature_extraction import MFCCFeatureExtraction
from load_file import LoadFile

In [3]:
tracks, labels = LoadFile('../assets/csv/test.csv').__getitem__()
test_features, test_labels = MFCCFeatureExtraction(tracks, labels).featuredata()
tracks, labels = LoadFile('../assets/csv/dev.csv').__getitem__()
dev_features, dev_labels = MFCCFeatureExtraction(tracks, labels).featuredata()
tracks, labels = LoadFile('../assets/csv/train.csv').__getitem__()
train_features, train_labels = MFCCFeatureExtraction(tracks, labels).featuredata()

In [4]:
dense_layer =  tf.keras.Sequential([
                                      Dense(100, input_shape =(20,) ),
                                      Activation('relu'),
                                      Dropout(0.5),

                                      Dense(200),
                                      Activation('relu'),
                                      Dropout(0.5),
                                    
                                      Dense(100),
                                      Activation('relu'),
                                      Dropout(0.5),
                                    
                                      Dense(5),
                                      Activation('softmax')
])

dense_layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [5]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

dense_layer.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
num_epochs = 90
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath = '../Checkpoints/audio_dense.hdf5', verbose = 1, save_best_only = True)

start = datetime.now()
dense_layer.fit(train_features, train_labels,batch_size = num_batch_size, epochs = num_epochs, validation_data = (dev_features, dev_labels), callbacks = [checkpointer])
end = datetime.now()

print("Training completed in time:", end - start)

Epoch 1/90
141/141 [==============================] - ETA: 0s - loss: 12.0727 - accuracy: 0.2447
Epoch 00001: val_loss improved from inf to 1.54357, saving model to ../Checkpoints/audio_dense.hdf5
141/141 [==============================] - 0s 2ms/step - loss: 12.0727 - accuracy: 0.2447 - val_loss: 1.5436 - val_accuracy: 0.3880
Epoch 2/90
138/141 [============================>.] - ETA: 0s - loss: 2.1559 - accuracy: 0.3372
Epoch 00002: val_loss improved from 1.54357 to 1.41463, saving model to ../Checkpoints/audio_dense.hdf5
141/141 [==============================] - 0s 908us/step - loss: 2.1484 - accuracy: 0.3376 - val_loss: 1.4146 - val_accuracy: 0.4200
Epoch 3/90
 73/141 [==============>...............] - ETA: 0s - loss: 1.4250 - accuracy: 0.4182
Epoch 00003: val_loss improved from 1.41463 to 0.96851, saving model to ../Checkpoints/audio_dense.hdf5
141/141 [==============================] - 0s 918us/step - loss: 1.3422 - accuracy: 0.4416 - val_loss: 0.9685 - val_accuracy: 0.5800
Epoch

In [6]:
# Dense layer
dense_layer.evaluate(test_features, test_labels, verbose = 1)

8/8 [==============================] - 0s 545us/step - loss: 0.1934 - accuracy: 0.9200


[0.19337405264377594, 0.9200000166893005]